<a href="https://colab.research.google.com/github/Craphtr/data-enginering-zoomcamp2025/blob/main/03-data-warehouse/dlt_workshop_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="alert alert-block" style = "background-color: black">,
        <p><b><font size="+4" color="orange">Homework Data Ingestion with dlt</font></b></p>
    </div>

In [4]:
!pip install dlt[bigquery]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.3/884.3 kB 29.1 MB/s eta 0:00:00


In [17]:
!pip install dlt[duckdb]

In [6]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator
import requests

#### __Q1.__

In [5]:
!dlt --version

dlt 1.6.1


#### __Q2. Define and Run Pipeline on NYC Taxi Dataset__

In [14]:
@dlt.resource(name="nyc_taxi_data")

def fetch_tripdata(service_type='yellow'):

    """Fetches NYC tripdata dynamically.
    - sevice_type: "yellow" or green"
    - Loops thru all years and months to fetch all available data
    """
    client = RESTClient(
        base_url = "https://us-central1-dlthub-analytics.cloudfunctions.net",
        #define pagination strategy - page-based pagination
        paginator = PageNumberPaginator(# pages numbered 1 to n
            base_page = 1, #start_page = 1
            total_path = None # No total count of pages known forehand, pagination ends when no more pages to process
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"): #API endpoint for retrieving data
        if not page:
          break
        yield page #yielding page by page exposes you endpoint data and prevents memory overflow

for page_data in fetch_tripdata():
    print(page_data)
    break

#Define and run the dlt pipeline
pipeline = dlt.pipeline(destination="duckdb")
load_info = pipeline.run(fetch_tripdata, write_disposition="replace")
print(load_info)



{'End_Lat': 40.742963, 'End_Lon': -73.980072, 'Fare_Amt': 45.0, 'Passenger_Count': 1, 'Payment_Type': 'Credit', 'Rate_Code': None, 'Start_Lat': 40.641525, 'Start_Lon': -73.787442, 'Tip_Amt': 9.0, 'Tolls_Amt': 4.15, 'Total_Amt': 58.15, 'Trip_Distance': 17.52, 'Trip_Dropoff_DateTime': '2009-06-14 23:48:00', 'Trip_Pickup_DateTime': '2009-06-14 23:23:00', 'mta_tax': None, 'store_and_forward': None, 'surcharge': 0.0, 'vendor_name': 'VTS'}
Pipeline dlt_colab_kernel_launcher load step completed in 2.93 seconds
1 load package(s) were loaded to destination duckdb and into dataset dlt_colab_kernel_launcher_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1739806791.6639392 is LOADED and contains no failed jobs


{'End_Lat': 40.742963, 'End_Lon': -73.980072, 'Fare_Amt': 45.0, 'Passenger_Count': 1, 'Payment_Type': 'Credit', 'Rate_Code': None, 'Start_Lat': 40.641525, 'Start_Lon': -73.787442, 'Tip_Amt': 9.0, 'Tolls_Amt': 4.15, 'Total_Amt': 58.15, 'Trip_Distance': 17.52, 'Trip_Dropoff_DateTime': '2009-06-14 23:48:00', 'Trip_Pickup_DateTime': '2009-06-14 23:23:00', 'mta_tax': None, 'store_and_forward': None, 'surcharge': 0.0, 'vendor_name': 'VTS'}


#### __Connecting `duckdb` to a database__

In [19]:

import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

#A database '<pipeline-duckdb>' was created in the current working directory to connect to it

#Connect to the duckdb database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

#Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

#Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,dlt_colab_kernel_launcher,dlt_colab_kernel_launcher_dataset,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,dlt_colab_kernel_launcher,dlt_colab_kernel_launcher_dataset,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,dlt_colab_kernel_launcher,dlt_colab_kernel_launcher_dataset,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,dlt_colab_kernel_launcher,dlt_colab_kernel_launcher_dataset,nyc_taxi_data,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


#### __ANS - `4 tables created`__

> Add blockquote



In [20]:
conn.sql("SELECT COUNT(*) FROM nyc_taxi_data")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        10000 │
└──────────────┘

#### __`ANS - 10,000 records extracted`__

#### __Q3. Explore the Dataset__

In [23]:
df = pipeline.dataset(dataset_type="default").nyc_taxi_data.df()

#### __Q4. Trip Duration Analysis__

In [26]:
with pipeline.sql_client() as client:
  res = client.execute_sql(
      """
      SELECT
      AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
      FROM nyc_taxi_data;
      """
  )
  print(res)

[(12.3049,)]
